In [1]:
import pandas as pd
from common import Data, Playoffs

In [2]:
df = pd.read_csv("data/2025.csv")

In [3]:
def get_standings(df):
    df["season"] = "2025"
    result = Data.get_nwsl_standings(df)
    result["games"] = result["wins"] + result["draws"] + result["losses"]
    result["max_points"] = result["points"] + 3 * (26 - result["games"])
    result = result[["team", "games", "points", "max_points"]].sort_values(["max_points"], ascending=False)
    result = result.reset_index(drop=True)
    result.index = result.index + 1
    return result


def add_results(df, scores):
    result = df.copy()

    for home, home_score, away, away_score in scores:
        result.loc[
            (df["home"] == home) & (df["away"] == away), ["home_score", "away_score"]
        ] = [home_score, away_score]

    return result


def get_matches_between(df, a, b):
    remaining = df[df["home_score"].isna()]
    count = len(
        remaining[
            ((remaining["home"] == a) & (remaining["away"] == b))
            | ((remaining["home"] == b) & (remaining["away"] == a))
        ].index
    )
    if count > 0:
        print(a, b, count)

In [4]:
new_schedule = add_results(
    df,
    [
        #("KC", 1, "CHI", 0),
        ("SD", 0, "ORL", 0),
        ("NJY", 1, "POR", 0),
        #("BFC", 0, "UTA", 0),
        ("LOU", 0, "LA", 0),
        ("WAS", 0, "HOU", 0),
        ("RGN", 1, "NC", 0)
    ],
)
get_standings(new_schedule)

,team,games,points,max_points
1,KC,21,53,68
2,WAS,22,38,50
3,NJY,22,34,46
4,RGN,22,33,45
5,SD,22,32,44
6,POR,22,31,43
7,ORL,22,30,42
8,NC,22,28,40
9,HOU,22,27,39
10,LOU,22,27,39


In [7]:
teams = [
    "NJY",
    "RGN",
    "SD",
    "POR",
    "ORL",
    "NC",
    "HOU",
    #"LOU",
]

for a in teams:
    for b in teams:
        if a > b:
            get_matches_between(new_schedule, a, b)

NJY NC 1
RGN NJY 1
RGN ORL 1
POR ORL 1
POR HOU 1
ORL HOU 1


In [53]:
teams = list(new_schedule["home"].unique())
g = Playoffs.calc_games_matrix(new_schedule, teams)
p0 = Playoffs.calc_initial_points(new_schedule, teams)
Playoffs.calculate_lowest_finish("KC", teams, g, p0)

8

In [25]:
df_2021 = Data.get_nwsl_matches()
df_2021 = df_2021[(df_2021["season"] == "2021") & (~df_2021["is_playoffs"])]
df_2021.loc[df_2021["date"] > "2021-10-13", ["home_score", "away_score"]] = None
get_standings(df_2021)

,team,games,points,max_points
1,POR,22,40,52
2,RGN,22,39,51
3,NJNY,20,29,47
4,WAS,22,33,45
5,NC,22,32,44
6,HOU,22,32,44
7,CHI,22,32,44
8,ORL,22,28,40
9,LOU,21,17,32
10,KCC,21,15,30


In [23]:
teams = ["CHI", "ORL", "WAS", "NJNY", "HOU", "NC"]

for a in teams:
    for b in teams:
        if a > b:
            get_matches_between(df_2021, a, b)

ORL CHI 2
WAS HOU 1
WAS NC 1
NJNY NC 1


In [ ]:
CHI and ORL both were missing tiebreaker against RGN

In [24]:
df_2021

,date,time,season,home,away,home_score,away_score,home_penalty_score,away_penalty_score,home_xg,away_xg,referee,attendance,stadium,is_forfeit,is_extra_time,is_pks,is_playoffs
805,2021-05-15,17:00,2024,LOU,KCC,0.0,0.0,0.0,0.0,0.8,0.5,Adorae Monroy,6622.0,Lynn Family Stadium,False,False,False,False
806,2021-05-15,19:00,2024,NJNY,HOU,1.0,0.0,0.0,0.0,0.7,2.1,Danielle Chesky,NaN,Red Bull Arena,False,False,False,False
807,2021-05-15,19:00,2024,RGN,NC,0.0,0.0,0.0,0.0,0.7,1.6,Karen Abt,2104.0,Cheney Stadium,False,False,False,False
808,2021-05-16,16:00,2024,POR,CHI,5.0,0.0,0.0,0.0,3.6,0.2,Katja Koroleva,NaN,Providence Park,False,False,False,False
809,2021-05-16,18:00,2024,ORL,WAS,1.0,1.0,0.0,0.0,1.4,2.1,Karen Callado,NaN,Exploria Stadium,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,2021-10-29,19:00,2024,ORL,CHI,NaN,NaN,0.0,0.0,0.2,1.1,Elvis Osmanovic,NaN,Exploria Stadium,False,False,False,False
921,2021-10-30,19:00,2024,POR,NC,NaN,NaN,0.0,0.0,1.8,1.4,Greg Dopka,17584.0,Providence Park,False,False,False,False
922,2021-10-30,20:00,2024,KCC,RGN,NaN,NaN,0.0,0.0,0.8,2.8,Laura Rodriguez,6345.0,Legends Field,False,False,False,False
923,2021-10-31,14:00,2024,WAS,HOU,NaN,NaN,0.0,0.0,1.4,0.6,Danielle Chesky,4542.0,Audi Field,False,False,False,False
